In [ ]:
#!pip install torch git+https://github.com/martijnvanbeers/transformers@feature/attention-transformers pandas seaborn matplotlib numpy scikit-learn spacy==2.3.7 https://github.com/explosion/spacy-models/releases/download/en_core_web_sm-2.3.1/en_core_web_sm-2.3.1.tar.gz
#!wget https://raw.githubusercontent.com/martijnvanbeers/nlp-attribution-notebooks/main/firsthalf.txt
#!wget https://raw.githubusercontent.com/martijnvanbeers/nlp-attribution-notebooks/main/valuezeroing.py

In [ ]:
import itertools
import numpy
import pandas
import seaborn
import matplotlib.pyplot as plt
import ipywidgets as widgets
import spacy
import torch

from transformers import (
    AutoConfig,
    AutoTokenizer,
    AutoModelForSequenceClassification, AutoModelForMaskedLM
)

from valuezeroing import calculate_scores

In [ ]:
## GPU
if torch.cuda.is_available():
    device = torch.device("cuda")
    print('We will use the GPU:', torch.cuda.get_device_name("cuda"))
else:
    device = torch.device("cpu")
    print('No GPU available, using the CPU instead.')



In [ ]:
corpus = pandas.read_csv("firsthalf.txt", sep="\t", header=None, names=["line"])

In [ ]:
with pandas.option_context("display.max_colwidth", 200):
    display(corpus.head(10))

In [ ]:
class TransformerTokenizer:
    def __init__(self, vocab, tokenizer):
        self.vocab = vocab
        self._tokenizer = tokenizer

    def __call__(self, text):
        result = self._tokenizer._tokenizer.encode(text)
        words = []
        spaces = []
        for wordix,g in itertools.groupby(zip(range(len(result.word_ids[1:-1])), result.word_ids[1:-1]), key=lambda t: t[1]):
            g = list(g)
            first_token = g[0][0]
            last_token = g[-1][0]
            start = result.offsets[first_token+1][0]
            end = result.offsets[last_token+1][1]
            words.append(text[start:end])
            if wordix < max(result.word_ids[1:-1]):
                # If next start != current end we assume a space in between
                next_start, next_end = result.offsets[last_token + 2]
                spaces.append(next_start > end)
            else:
                if end < len(text):
                    spaces.append(True)
                else:
                    spaces.append(False)
        return spacy.tokens.Doc(self.vocab, words=words, spaces=spaces)

In [ ]:
transformer = "bert-base-uncased"
config = AutoConfig.from_pretrained(transformer, output_attentions=True)#, attentions_with_qk=True)
tokenizer = AutoTokenizer.from_pretrained(transformer)
model = AutoModelForMaskedLM.from_pretrained(transformer, config=config)
model.to(device)
model.eval()

nlp = spacy.load("en_core_web_sm")
nlp.tokenizer = TransformerTokenizer(nlp.vocab, tokenizer)

In [ ]:
#poslist = ["[CLS]", "[SEP]", "CCONJ", "PROPN", "PRON", "AUX", "VERB", "ADP", "NOUN", "SYM", "NUM", "DET", "PUNCT"]
poslist = [
    "SELF",
    "[CLS]",
    "[SEP]",
#    "",
    "ADJ",
    "ADP",
    "ADV",
    "AUX",
    "CONJ",
    "CCONJ",
    "DET",
    "INTJ",
    "NOUN",
    "NUM",
    "PART",
    "PRON",
    "PROPN",
    "PUNCT",
    "SCONJ",
    "SYM",
    "VERB",
    "X",
    "EOL",
    "SPACE",
]

In [ ]:
len(poslist)

In [ ]:
combined_df = None
token_count = 0
for i, row in corpus.head(10).iterrows():
    doc = nlp(row['line'])
    scores_matrix, rollout_matrix, att_matrix = calculate_scores(config, model, "bert", tokenizer, doc.text)
    att_matrix = att_matrix.detach().cpu().numpy()
    token_count += scores_matrix.shape[-1]
    result = tokenizer(doc.text, return_special_tokens_mask=True, return_offsets_mapping=True)
    all_tokens = result.tokens()
    docpos = ["[CLS]"] + [doc[t].pos_ for t in result.word_ids()[1:-1]] + ["[SEP]"]
    index = pandas.MultiIndex.from_product(
            [numpy.arange(12)+1, numpy.arange(12)+1, all_tokens, all_tokens],
            names=['layer','head','from', 'to']
        )
    score_df = pandas.DataFrame(
            numpy.hstack([
                    scores_matrix.reshape(-1, 1),
                    rollout_matrix.reshape(-1,1),
                    att_matrix.reshape(-1,1)
                ]),
            index=index,
            columns=["valuezeroing", "rollout_vz", "raw_attention"]
        ).reset_index()
    score_df['from_pos'] = pandas.Categorical(
            numpy.tile(numpy.repeat(numpy.array(docpos), len(all_tokens)), 12*12),
            categories=poslist
        )
    score_df['to_pos'] = pandas.Categorical(
            numpy.tile(numpy.array(docpos), len(all_tokens)*12*12),
            categories=poslist
        )
    score_df['from_ix'] = numpy.tile(numpy.repeat(numpy.arange(len(all_tokens)), len(all_tokens)), 12*12)
    score_df['to_ix'] = numpy.tile(numpy.arange(len(all_tokens)), len(all_tokens)*12*12)
    score_df['to_pos'] = score_df.apply(lambda r: "SELF" if r['from_ix'] == r['to_ix'] else r['to_pos'], axis=1)
    score_df['sent'] = i
    counts = ((score_df[(score_df['layer'] == 1) & (score_df['head'] == 1)]
                    .groupby(["from_pos", "to_pos"])
                    .agg({"from": "count"}))
                    .rename(columns={'from': 'combo_count'})
                    .reset_index()
            )
    score_df = score_df.merge(counts, how="left", on=["from_pos", "to_pos"])
    if combined_df is None:
        combined_df = score_df
    else:
        combined_df = pandas.concat([combined_df, score_df])

In [ ]:
combined_df.iloc[:50,:]

In [ ]:
with pandas.option_context("display.max_rows", None):
    display(
        combined_df[
            (combined_df['layer'] == 1) &
            (combined_df['head'] == 1) &
            (combined_df['from_pos'] == "NOUN") &
            (combined_df['to_pos'] == "ADJ")
        ]
    )

In [ ]:
combined_df[
        (combined_df['layer'] == 3) &
        (combined_df['head'] == 1) &
        (combined_df['sent'] == 0) &
        (combined_df['from_pos'] == "ADJ") &
        (combined_df['to_pos'] == "ADJ")
    ]

In [ ]:
g = (combined_df
     .groupby(["layer", "head", "from_pos", "to_pos"])
     .agg({
             "raw_attention": lambda n: numpy.sum(n) / token_count,
             "valuezeroing": lambda n: numpy.sum(n) / token_count,
             "rollout_vz": lambda n: numpy.sum(n) / token_count,
         })
     .dropna()
     .reset_index())
 

In [ ]:
combined_df['adjusted_attention'] = combined_df['raw_attention'] / combined_df['combo_count']
combined_df['adjusted_vz'] = combined_df['valuezeroing'] / combined_df['combo_count']
combined_df['adjusted_rollout_vz'] = combined_df['rollout_vz'] / combined_df['combo_count']


In [ ]:
ga = (combined_df
     .groupby(["layer", "head", "from_pos", "to_pos"])
     .agg({
         "adjusted_attention": lambda n: numpy.sum(n) / token_count,
         "adjusted_vz": lambda n: numpy.sum(n) / token_count,
         "adjusted_rollout_vz": lambda n: numpy.sum(n) / token_count,
        })
     .dropna()
     .reset_index())
 

In [ ]:
def show_head(ignores=[], sortby="valuezeroing", layer=1, head=1, top_n=5):
    am = {
        'raw_attention': "adjusted_attention", 
        'valuezeroing': "adjusted_vz",
        'rollout_vz': "adjusted_rollout_vz",
    }
    display(g[~g['from_pos'].isin(ignores) & ~g['to_pos'].isin(ignores) & (g['layer'] == layer) & (g['head'] == head)].sort_values(sortby, ascending=False).head(top_n))
    display(ga[~ga['from_pos'].isin(ignores) & ~ga['to_pos'].isin(ignores) & (ga['layer'] == layer) & (ga['head'] == head)].sort_values(am[sortby], ascending=False).head(top_n))

In [ ]:
w = widgets.interactive(show_head,
                ignores=widgets.SelectMultiple(
                        options=poslist,
                        value=['[CLS]', '[SEP]'],
                        description='Ignored POS',
                        rows=25,
                        disabled=False
                    ),
                sortby=widgets.RadioButtons(
                        options=['raw_attention', 'valuezeroing', 'rollout_vz'],
                        value='valuezeroing',
                        layout={'width': 'max-content'}, # If the items' names are long
                        description='sort by',
                    ),
                layer=widgets.IntSlider(min=1, max=12, value=1, step=1),
                head=widgets.IntSlider(min=1, max=12, value=1, step=1),
                top_n=widgets.IntSlider(min=3, max=20, value=10, step=1)
            )
display(w)

In [ ]:
def show_combo(from_pos, to_pos, sortby):
    with pandas.option_context("display.max_rows", 150):
        display(
            pandas.concat([
                g[(g['from_pos'] == from_pos) & (g['to_pos'] == to_pos)],
                ga[(ga['from_pos'] == from_pos) & (ga['to_pos'] == to_pos)][['adjusted_attention', 'adjusted_vz', 'adjusted_rollout_vz']]
            ], axis=1).reset_index(drop=True).sort_values(sortby, ascending=False)
        )

In [ ]:
w = widgets.interactive(show_combo,
                from_pos=widgets.Select(
                        options=poslist,
                        value='NOUN',
                    ),
                to_pos=widgets.Select(
                        options=poslist,
                        value='NOUN',
                    ),
                sortby=widgets.RadioButtons(
                        options=['raw_attention', 'valuezeroing', 'rollout_vz', 'adjusted_attention', 'adjusted_vz', 'adjusted_rollout_vz'],
                        value='valuezeroing',
                        layout={'width': 'max-content'}, # If the items' names are long
                        description='sort by',
                    ),

            )
display(w)